# Costco Label OBB Detection — Training on Colab

This notebook trains a YOLO11-OBB model to detect Costco price labels.

**Runtime → Change runtime type → T4 GPU** before running.

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repo
!git clone https://github.com/ankurkaushal/LabelParser.git
%cd LabelParser

In [ ]:
# Install dependencies
!pip install -q ultralytics albumentations

In [ ]:
from pathlib import Path
from ultralytics import YOLO

ROOT = Path.cwd()

# Verify dataset
train_imgs = list((ROOT / "dataset" / "images" / "train").glob("*.*"))
test_imgs = list((ROOT / "dataset" / "images" / "test").glob("*.*"))
print(f"Train images: {len(train_imgs)}, Test images: {len(test_imgs)}")

In [ ]:
# Train
model = YOLO("yolo11n-obb.pt")

results = model.train(
    data=str(ROOT / "dataset.yaml"),
    epochs=100,
    imgsz=640,
    batch=16,         # T4 has 16GB VRAM — can handle batch=16
    patience=20,
    device=0,         # GPU 0
    project=str(ROOT / "runs"),
    name="costco_label_obb",
    exist_ok=True,
    # Augmentation tuned for small dataset
    mosaic=1.0,
    flipud=0.5,
    fliplr=0.5,
    degrees=15.0,
    scale=0.5,
    translate=0.1,
)

print(f"\nBest model: {ROOT / 'runs' / 'costco_label_obb' / 'weights' / 'best.pt'}")

In [ ]:
# View training curves
from IPython.display import Image, display

results_img = ROOT / "runs" / "costco_label_obb" / "results.png"
if results_img.exists():
    display(Image(filename=str(results_img), width=800))

In [ ]:
# Validate on test set
best = YOLO(str(ROOT / "runs" / "costco_label_obb" / "weights" / "best.pt"))
metrics = best.val(data=str(ROOT / "dataset.yaml"), device=0)
print(f"\nmAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# Run predictions on a few test images
import random

test_dir = ROOT / "dataset" / "images" / "test"
sample_imgs = random.sample(list(test_dir.glob("*.*")), min(6, len(list(test_dir.glob("*.*")))))

preds = best.predict(source=sample_imgs, imgsz=640, device=0, save=True,
                      project=str(ROOT / "runs"), name="predict_samples", exist_ok=True)

# Display predictions
pred_dir = ROOT / "runs" / "predict_samples"
for img_path in sorted(pred_dir.glob("*.jpg"))[:6]:
    display(Image(filename=str(img_path), width=500))
    print(img_path.name)
    print("---")

In [ ]:
# Download the best model
from google.colab import files

best_pt = ROOT / "runs" / "costco_label_obb" / "weights" / "best.pt"
files.download(str(best_pt))